# Fase 2 – MLOps: Rol de Data Scientist
**Autor:** Ricardo Miguel Aguilar Rosas  
**Matrícula:** A01171223  
**Equipo:** 8 – Energy Efficiency Prediction  
**Fecha:** Octubre 2025


In [28]:
!pip install -q mlflow==2.15.0

In [29]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:

%cd /content
!git clone https://github.com/PosgradoMNA/TC5044.10-Equipo-8.git
%cd TC5044.10-Equipo-8
!git checkout ricardo-datascientist

/content
fatal: destination path 'TC5044.10-Equipo-8' already exists and is not an empty directory.
/content/TC5044.10-Equipo-8
Already on 'ricardo-datascientist'
Your branch is up to date with 'origin/ricardo-datascientist'.


In [31]:
# (Opcional) ver qué hay
!ls -l src/data

# Copiar desde tu Drive si hace falta (ajusta la ruta si tu carpeta se llama distinto)
!cp "/content/drive/MyDrive/MNA/MLOps/energy_efficiency_modified.csv" "src/data/energy_efficiency_modified.csv"

# Confirmar
!ls -l src/data | sed -n '1,50p'


total 52
drwxr-xr-x 2 root root  4096 Oct 29 22:33 cleansed
-rw------- 1 root root 43747 Oct 29 22:43 energy_efficiency_modified.csv
-rw-r--r-- 1 root root   111 Oct 29 22:33 energy_efficiency_modified.csv.dvc
total 52
drwxr-xr-x 2 root root  4096 Oct 29 22:33 cleansed
-rw------- 1 root root 43747 Oct 29 23:32 energy_efficiency_modified.csv
-rw-r--r-- 1 root root   111 Oct 29 22:33 energy_efficiency_modified.csv.dvc


In [32]:
import sys, os
repo = '/content/TC5044.10-Equipo-8'
if repo not in sys.path: sys.path.append(repo)         # raiz del repo
srcp = f'{repo}/src'
if srcp not in sys.path: sys.path.append(srcp)         # carpeta src


In [33]:
!sed -n '1,160p' src/handlers/data_loader.py


import pandas as pd
from pathlib import Path
import subprocess


class DataLoader:
    NAMING_MAP = {
        "X1": "relative_compactness",
        "X2": "surface_area",
        "X3": "wall_area",
        "X4": "roof_area",
        "X5": "overall_height",
        "X6": "orientation",
        "X7": "glazing_area",
        "X8": "glazing_area_distribution",
        "Y1": "heating_load",
        "Y2": "cooling_load",
    }

    def getBaseDir(self):
        """
        Get the base directory path for the current file.
        
        Returns the parent directory of the current file as a Path object.
        """
        return Path(__file__).resolve().parent

    def getDataFrameFromFile(self, route_to_file: str):
        """
        Load a CSV file and return a pandas DataFrame with renamed columns.
        
        Keyword arguments:
        route_to_file -- relative path to the CSV file from the base directory
        
        Returns a DataFrame with columns renamed according to NAMIN

In [34]:
# 1️⃣ IMPORTA LA CLASE ANTES DE USARLA
from src.handlers.data_loader import DataLoader

# 2️⃣ LUEGO YA PUEDES CREAR LA INSTANCIA Y CARGAR TU CSV
from pathlib import Path
data_path = Path("data/energy_efficiency_modified.csv")

loader = DataLoader()
df = loader.getDataFrameFromFile(data_path)

df.head()




Succesfully loaded DF from /content/TC5044.10-Equipo-8/src/data/energy_efficiency_modified.csv... 



,relative_compactness,surface_area,wall_area,roof_area,overall_height,orientation,glazing_area,glazing_area_distribution,heating_load,cooling_load,mixed_type_col
0,0.98,514.5,294.0,110.25,7.0,2.0,0.0,0.0,15.55,21.33,637
1,0.98,514.5,294.0,110.25,7.0,3.0,0.0,0.0,15.55,21.33,618
2,0.98,514.5,294.0,110.25,7.0,4.0,0.0,0.0,15.55,21.33,412
3,0.98,514.5,294.0,110.25,7.0,5.0,0.0,0.0,15.55,21.33,243
4,0.9,563.5,318.5,122.5,7.0,2.0,0.0,0.0,20.84,28.28,979


In [35]:
from sklearn.model_selection import train_test_split

# Elegimos 'heating_load' como variable objetivo
target_col = "heating_load"

X = df.drop(columns=[target_col])
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

len(X_train), len(X_test)


(626, 157)

In [36]:
# 1) importar la clase
from src.handlers.data_preprocessor import DataPreprocessor

# 2) instanciar con tu DataFrame original
pre = DataPreprocessor(df)

# 3) ejecutar los pasos que sí existen en esa clase
pre.convert_numeric()
pre.impute_missing()
pre.detect_outliers()
pre.standardize()

# 4) obtener el DataFrame ya transformado
df_preprocessed = pre.df

# 5) checar rápido
print(df_preprocessed.shape)
df_preprocessed.head()




Converting numeric values... 


Initializing imputation of values... 


Missing values before imputation: 347 


Missing values after imputation: 0 


Initializing outlier analysis... 


Rows detected as outliers: 76 


Outliers removed 


Standardized numeric columns using StandardScaler for the following columns: ['relative_compactness', 'surface_area', 'wall_area', 'roof_area', 'overall_height', 'orientation', 'glazing_area', 'glazing_area_distribution', 'heating_load', 'cooling_load', 'mixed_type_col'] 

(707, 11)


,relative_compactness,surface_area,wall_area,roof_area,overall_height,orientation,glazing_area,glazing_area_distribution,heating_load,cooling_load,mixed_type_col
0,2.063837,-1.799068,-0.568198,-1.475885,0.965243,-1.366685,-1.782606,-1.802798,-0.681674,-0.364026,0.595668
1,2.063837,-1.799068,-0.568198,-1.475885,0.965243,-0.469520,-1.782606,-1.802798,-0.681674,-0.364026,0.519367
2,2.063837,-1.799068,-0.568198,-1.475885,0.965243,0.427644,-1.782606,-1.802798,-0.681674,-0.364026,-0.307896
3,2.063837,-1.799068,-0.568198,-1.475885,0.965243,1.324809,-1.782606,-1.802798,-0.681674,-0.364026,-0.986574
4,1.297083,-1.235139,-0.003990,-1.204935,0.965243,-1.366685,-1.782606,-1.802798,-0.159847,0.363192,1.969086


In [37]:
# Ver el contenido del archivo (las primeras 200 líneas)
!sed -n '1,200p' src/handlers/data_preprocessor.py


import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


class DataPreprocessor:
    NUMERIC_COLS = [
        "relative_compactness",
        "surface_area",
        "wall_area",
        "roof_area",
        "overall_height",
        "orientation",
        "glazing_area",
        "glazing_area_distribution",
        "heating_load",
        "cooling_load",
        "mixed_type_col",
    ]

    def __init__(self, df):
        self.df = df
        self.outliers = []

    def convert_numeric(self):
        """
        Convert specified columns to numeric float type.
        
        Converts all columns in NUMERIC_COLS to float, handling errors by coercing to NaN.
        """
        print(f"\nConverting numeric values...", "\n")
        for col in self.NUMERIC_COLS:
            self.df[col] = pd.to_numeric(self.df[col], errors="coerce").astype(float)

    def impute_missing(self):
        """
        Fill missing values in numeric columns with their median 

In [38]:
# 2. PREPROCESAMIENTO (usa los métodos que realmente existen en tu clase)
from src.handlers.data_preprocessor import DataPreprocessor

pre = DataPreprocessor(df)          # <- tu __init__ requiere df
pre.convert_numeric()               # convierte a float las columnas NUMERIC_COLS
pre.impute_missing()                # imputación por mediana
pre.detect_outliers()               # quita outliers por IQR
pre.standardize()                   # estandariza con StandardScaler

df_pre = pre.df                     # dataframe ya preprocesado
df_pre.head()



Converting numeric values... 


Initializing imputation of values... 


Missing values before imputation: 0 


Missing values after imputation: 0 


Initializing outlier analysis... 


Rows detected as outliers: 0 


Outliers removed 


Standardized numeric columns using StandardScaler for the following columns: ['relative_compactness', 'surface_area', 'wall_area', 'roof_area', 'overall_height', 'orientation', 'glazing_area', 'glazing_area_distribution', 'heating_load', 'cooling_load', 'mixed_type_col'] 



,relative_compactness,surface_area,wall_area,roof_area,overall_height,orientation,glazing_area,glazing_area_distribution,heating_load,cooling_load,mixed_type_col
0,2.063837,-1.799068,-0.568198,-1.475885,0.965243,-1.366685,-1.782606,-1.802798,-0.681674,-0.364026,0.595668
1,2.063837,-1.799068,-0.568198,-1.475885,0.965243,-0.469520,-1.782606,-1.802798,-0.681674,-0.364026,0.519367
2,2.063837,-1.799068,-0.568198,-1.475885,0.965243,0.427644,-1.782606,-1.802798,-0.681674,-0.364026,-0.307896
3,2.063837,-1.799068,-0.568198,-1.475885,0.965243,1.324809,-1.782606,-1.802798,-0.681674,-0.364026,-0.986574
4,1.297083,-1.235139,-0.003990,-1.204935,0.965243,-1.366685,-1.782606,-1.802798,-0.159847,0.363192,1.969086


In [39]:
from sklearn.model_selection import train_test_split

TARGET_COL = "heating_load"  # ajusta si tu target es distinto
X = df_pre.drop(columns=[TARGET_COL])
y = df_pre[TARGET_COL]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
len(X_train), len(X_test)


(565, 142)

In [40]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import mlflow, mlflow.sklearn

results = []

# === Evaluación manual + logging a MLflow ===
for name, model in trainer.models.items():
    parts = name.split("_")                     # ← definimos 'parts' aquí
    target = "_".join(parts[-2:])               # heating_load o cooling_load

    X_test = trainer.X_test_scaled
    y_test = trainer.Y_test[target]
    preds = model.predict(X_test)

    # Métricas
    r2   = r2_score(y_test, preds)
    mae  = mean_absolute_error(y_test, preds)
    rmse = mean_squared_error(y_test, preds, squared=False)

    # Logging en MLflow
    with mlflow.start_run(run_name=name):
        mlflow.log_param("algorithm", "_".join(parts[:-2]))  # LinearRegression, RandomForest, etc.
        mlflow.log_param("target", target)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("rmse", rmse)
        mlflow.sklearn.log_model(model, artifact_path="model")

    results.append((name, r2, mae, rmse))
    print(f"{name} ✅ R2={r2:.3f} | MAE={mae:.3f} | RMSE={rmse:.3f}")

print("\n✅ Todos los modelos fueron evaluados y registrados en MLflow.")
results




/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2025/10/29 23:32:52 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


LinearRegression_heating_load ✅ R2=0.911 | MAE=0.179 | RMSE=0.272


2025/10/29 23:32:56 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


LinearRegression_cooling_load ✅ R2=0.850 | MAE=0.250 | RMSE=0.356


2025/10/29 23:32:59 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


...

2025/10/29 23:33:04 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


GradientBoosting_heating_load ✅ R2=0.953 | MAE=0.070 | RMSE=0.198


2025/10/29 23:33:06 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


GradientBoosting_cooling_load ✅ R2=0.923 | MAE=0.121 | RMSE=0.255

✅ Todos los modelos fueron evaluados y registrados en MLflow.


[('LinearRegression_heating_load',
  0.9114839904840489,
  0.17908273616179302,
  0.2715086226353903),
 ('LinearRegression_cooling_load',
  0.8498590352644076,
  0.2499870907776117,
  0.35609188431465283),
 ('RandomForest_heating_load',
  0.9578185382479615,
  0.07137492712741686,
  0.18742759218655583),
 ('RandomForest_cooling_load',
  0.9279893537554069,
  0.12495638902257658,
  0.24661008276142313),
 ('GradientBoosting_heating_load',
  0.9528949783887783,
  0.06957749544188331,
  0.19806435174246106),
 ('GradientBoosting_cooling_load',
  0.9227084604342437,
  0.12115738096137918,
  0.2554926752251416)]

In [41]:
!sed -n '1,200p' src/handlers/model_trainer.py


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


class ModelTrainer:
    def __init__(
        self,
        df,
        target_cols=["heating_load", "cooling_load"],
        test_size=0.2,
        random_state=42,
    ):
        self.df = df.copy()
        self.target_cols = target_cols
        self.test_size = test_size
        self.random_state = random_state
        self.X_train = self.X_test = self.Y_train = self.Y_test = None
        self.scaler = StandardScaler()
        self.models = {}

    def list_models(self):
        """
        Print the names of all currently available models for training.
        """
        print("\nCurrent models to train:")
        for name in self.models.keys():
            print(f"- {name}")

    def split_and_scale(self):
        """
        Split the dataset into tra